In [4]:
import numpy as np
from numpy.random import default_rng
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import PIL
import pandas as pd
import matplotlib.pyplot as plt

In [45]:
filepath='.'
filedir=os.path.join(os.getcwd(),filepath,'Vegetable Images')
columns=['img_name','label']
X_train=[]
y_train=[]
X_test=[]
y_test=[]
X_valid=[]
y_valid=[]

for i in os.listdir(filedir):
    ifiledir=os.path.join(filedir,i)
    for iter,j in enumerate(os.listdir(ifiledir)):
        jfiledir=os.path.join(ifiledir,j)
        for k in os.listdir(jfiledir):
            kfiledir=os.path.join(jfiledir,k)
            if(i=='train'):            
                X_train.append(np.array(PIL.Image.open(kfiledir)))      
                y_train.append(iter)
            if(i=='test'):            
                X_test.append(np.array(PIL.Image.open(kfiledir)))       
                y_test.append(iter)
            if(i=='validation'):            
                X_valid.append(np.array(PIL.Image.open(kfiledir)))       
                y_valid.append(iter)

In [46]:
#X_train = np.array(X_train)
to_remove = []
for ind, image in enumerate(X_train):
    if len(image) != 224:
        print(f'incorrect size in train position: {ind}')
        print(f'size is {len(image)}')
        to_remove.append(ind)

for ind, i in enumerate(to_remove):
    del X_train[i-ind]
    del y_train[i-ind]
    print(f'removed train {i}')

to_remove = []
for ind, image in enumerate(X_test):
    if len(image) != 224:
        print(f'incorrect size in test position: {ind}')
        print(f'size is {len(image)}')
        to_remove.append(ind)

for ind, i in enumerate(to_remove):
    del X_train[i-ind]
    del y_train[i-ind]
    print(f'removed test {i}')

to_remove = []
for ind, image in enumerate(X_valid):
    if len(image) != 224:
        print(f'incorrect size in valid position: {ind}')
        print(f'size is {len(image)}')
        to_remove.append(ind)

for ind, i in enumerate(to_remove):
    del X_train[i-ind]
    del y_train[i-ind]
    print(f'removed valid {i}')

for ind, image in enumerate(X_train):
    if len(image) != 224:
        print(f'incorrect size in train position: {ind}')
        print(f'size is {len(image)}')

for ind, image in enumerate(X_test):
    if len(image) != 224:
        print(f'incorrect size in test position: {ind}')
        print(f'size is {len(image)}')

for ind, image in enumerate(X_valid):
    if len(image) != 224:
        print(f'incorrect size in valid position: {ind}')
        print(f'size is {len(image)}')

incorrect size in train position: 1429
size is 193
incorrect size in train position: 1525
size is 205
incorrect size in train position: 1608
size is 200
incorrect size in train position: 10125
size is 211
incorrect size in train position: 10175
size is 198
incorrect size in train position: 10740
size is 210
removed train 1429
removed train 1525
removed train 1608
removed train 10125
removed train 10175
removed train 10740
incorrect size in test position: 2048
size is 207
removed test 2048
incorrect size in valid position: 2140
size is 187
incorrect size in valid position: 2152
size is 223
removed valid 2140
removed valid 2152
incorrect size in test position: 2048
size is 207
incorrect size in valid position: 2140
size is 187
incorrect size in valid position: 2152
size is 223


# Base Model

In [ ]:
class myNN(nn.Module):
    def __init__(self):
        super().__init__()